## Preprocessing
- This notebook will walk through the steps of taking the raw text files provided by kaggle, formatting it to remove irrelevant data, and then changing the structure of the data so that it can be used in a recommender system.

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from collections import deque
import io

Because the data is provided in format that is difficult to work with, we have to format it so that we are able to both visualize it better, and work with it easier. This function goes through the following steps to do this
- Step 1: This step reads the text files given a specific path, and assigns the columns names
- Step 2: This step find the rows in the data frame where the rating column is missing. This is done because where the rows where the review is missing, there is the movie_id number. A list is created that contains the index of where the movie_id is, as well the movie_id itself.
- Step 3: This step shift the entire the entire list of movie_id's and their indexes down by one.
- Step 4: This step looks at the original list of movie_id's and their indexes, the shifted list of movie_id's and their indexes, makes sure that each movie_id is assigned to a copy of the data frame as long as they are between the two movie_id indexes, and then creates a new column of the actual movie id itself. The end is a single data frame for each movie of all the reviews for each movie, and the movie_id being a new column
- Step 5: This final step then takes the list of the data frames created in step 4, combines them all together, print that the formatting is done, and returns the final, formatted data frame.

In [2]:
def formatting(path):
    #Step 1
    df_raw = pd.read_csv(path, header=None, names=['user_id', 'rating', 'date'], usecols=[0, 1, 2])
    #Step 2
    tmp_movies = df_raw[df_raw['rating'].isna()]['user_id'].reset_index()
    movie_indices = [[index, int(movie[:-1])] for index, movie in tmp_movies.values]
    #Step 3
    shifted_movie_indices = deque(movie_indices)
    shifted_movie_indices.rotate(-1)
    #Step 4
    user_data = []
    for [df_id_1, movie_id], [df_id_2, next_movie_id] in zip(movie_indices, shifted_movie_indices):
        if df_id_1<df_id_2:
            tmp_df = df_raw.loc[df_id_1+1:df_id_2-1].copy()
        else:
            tmp_df = df_raw.loc[df_id_1+1:].copy()
        tmp_df['movie'] = movie_id
        user_data.append(tmp_df)
    #Step 5
    df = pd.concat(user_data)
    print('done formatting')
    return df

- The data provided is split into 4 separate text files, so the function created above will be run for all 4 text files.

In [3]:
df1 = formatting('../Data/combined_data_1.txt')
df2 = formatting('../Data/combined_data_2.txt')
df3 = formatting('../Data/combined_data_3.txt')
df4 = formatting('../Data/combined_data_4.txt')

done formatting
done formatting
done formatting
done formatting


- Because our data set is so large, we have to filter it to only contain users and movies that have a significant number of reviews. The goal is to only recommend movies that have proven their quality with many reviews, and to user reviews who have clear preferences of movies to recommend with. This function does this by grouping for every variable for a column, and then determining how many instances each individual variable has, and only returns that ones that have more instances than the provided threshold

In [4]:
def drop_threshold(threshold,column,df):
    filtered_df = df[df.groupby(column)[column].transform('size') >= threshold]
    print('This function dropped',len(df)-len(filtered_df),'rows by filtering on the',column,'column')
    return filtered_df

This for loop applies the function created above to each of of the 4 data frames. 
- Step 1: The for loop first drops the Date column, as it will not be used in recommending reviews to others. 
- Step 2: This step applies the function above to each data frame, and drops users that have less than 150 reviews, and drops movies that have less than 15,000 reviews
After the for loop finishes, It is important to delete the old data frames from memory. This is done because the files that are being processed are somewhat large, and this step helps preserve RAM.

In [5]:
df_list = []
i = 1
for df in [df1,df2,df3,df4]:
    print('For Dataframe',i,':')
    #Step 1
    df.drop(columns=['date'],inplace = True)
    #Step 2
    dropped_user = drop_threshold(150,'user_id',df)
    df_list.append(drop_threshold(15000,'movie',dropped_user))
    i += 1
del df1
del df2
del df3
del df4

For Dataframe 1 :
This function dropped 14870913 rows by filtering on the user_id column
This function dropped 5612461 rows by filtering on the movie column
For Dataframe 2 :
This function dropped 15290681 rows by filtering on the user_id column
This function dropped 6251804 rows by filtering on the movie column
For Dataframe 3 :
This function dropped 14808080 rows by filtering on the user_id column
This function dropped 5096893 rows by filtering on the movie column
For Dataframe 4 :
This function dropped 15757559 rows by filtering on the user_id column
This function dropped 5627403 rows by filtering on the movie column


- This cell combines all 4 of the filtered data frames into one large data frames, and assigns the proper data type to the user_id column. This cell also delete the list of the filtered data frames in order to preserve RAM

In [6]:
user_df = pd.concat(df_list)
user_df['user_id'] = pd.to_numeric(user_df['user_id'])
del df_list

- This cell converts the provided movie csv, which consists of the movie title, which year it was released, and its corresponding movie id value, into a pandas data frame. It then creates a dictionary linking all of the movie titles to their respective movie id's, and then replacing the movie ids from the review data frame with the movie title.

In [7]:
movies = pd.read_csv("../Data/movie_titles.csv",encoding = 'latin1')
movie_dict = dict(zip(movies['movie_id'],movies['title']))
del movies
user_df['movie'] = user_df['movie'].map(movie_dict)

- This cell pivots our review data frame so that each user is their own row, each movie is its own column, and the values being each individuals score for the movie is they did review it. If a user did note review a particular user, than that value will be indicated with a NA, or null value

In [8]:
data = user_df.pivot_table(values='rating',index='user_id',columns='movie')
del user_df

- This cell removes each users bias from the reviews. What this means in context is that for some users, they will rate every movie they enjoy as a 4, and the ones they really enjoy as a 5, while some will rate movies they enjoy as a 3, and the ones they really enjoy as a 4 or 5. By subtracting the average review from each review, we are able to the if an individual user enjoyed a particular movie more or less than others.

In [9]:
means = np.mean(data,axis = 1)
for row in data.index:
    data.loc[row] = data.loc[row] - means[row]

- This cell repeats the process above, but for movies. For example, a movie like star wars may have a lot of 5 star reviews. If an individual person were to rate it 5 stars, that doesn't mean a lot. If a user were to rate a movie higher than the average rating for the movie, that would indicate that particular movie was especially appealing to that user when compared to others. 

In [10]:
col_means = np.mean(data,axis = 0)
for movie in data.columns:
    data[movie] = data[movie] - col_means[movie]

- By filling the missing reviews with 0, we imply that there are no positive or negative association with that particular movie

In [11]:
data.fillna(0,inplace= True)

- This function's purpose is to convert a pandas data frame into a csv file, and then copying that file into a SQL database. This function is much more efficient than the default pandas function ```df.to_sql```. This function was created by stack overflow user mgoldwasser. The post of this code can be found at https://stackoverflow.com/questions/31997859/bulk-insert-a-pandas-dataframe-using-sqlalchemy/33529549

In [12]:
def write_to_table(df, db_engine, schema, table_name, if_exists='replace'):
    data.reset_index(inplace = True)
    string_data_io = io.StringIO()
    df.to_csv(string_data_io, sep='|', index=False)
    pd_sql_engine = pd.io.sql.pandasSQL_builder(db_engine, schema=schema)
    table = pd.io.sql.SQLTable(table_name, pd_sql_engine, frame=df,
                               index=False, if_exists=if_exists, schema=schema)
    table.create()
    string_data_io.seek(0)
    string_data_io.readline()
    with db_engine.connect() as connection:
        with connection.connection.cursor() as cursor:
            copy_cmd = "COPY %s.%s FROM STDIN HEADER DELIMITER '|' CSV" % (schema, table_name)
            cursor.copy_expert(copy_cmd, string_data_io)
        connection.connection.commit()

- This cell is creating the engine, or the connection, with the postgreSQL server created on an AWS instance
- Note: The password and IP_address are not provided in this notebook. If you want to run this locally, I recommend that you save the ```data``` data frame to a csv file.

In [13]:
engine = create_engine('postgres://postgres:password@ip_address:5432/postgres')

- This final cell applies the function above, and defines the table name as raw_user

In [14]:
write_to_table(data,engine,'public','raw_user')